In [ ]:
import sys
from datetime import datetime
import requests
import json
import datetime
import pandas as pd

# Mac
output_path = '/Users/adamfifield/Library/CloudStorage/OneDrive-UNHCR/00 SharePoint Sites/Documents - Greening and Sustainability Team/Green Data Team/01 Data Sources/01 Greenbox/01 EyeDro API Data/03 Verbose Meter Details/'
output_file = f"Meter_Inventory_Verbose{datetime.datetime.today().strftime('%Y-%m-%d')}.xlsx"

# Windows
# output_path = r'C:\Users\FIFIELD\OneDrive - UNHCR\00 SharePoint Sites\Documents - Greening and Sustainability Team\Green Data Team\01 Data Sources\01 Greenbox\01 EyeDro API Data\03 Verbose Meter Details'
# output_full_path = output_path+'\\'+output_file

output_full_path = output_path+output_file

output_log_path = output_path + "99 Runtime Logs/" + f"{datetime.datetime.today().strftime('%Y-%m-%d')}_log.txt"
sys.stdout = open(output_log_path,'wt')

print('Libraries Imported, Output Paths Set')

inventory_url_base = "https://api.eyedro.com/customcmd?&Cmd=Unhcr.GetDeviceInventoryList&UserKey=" 
inventory_key = "UNHCRMHiYgbHda9cRv4DuPp28DnAnfeV8s6umP5R"

print('API URL and Key Set')

inv_resp = requests.get(inventory_url_base+inventory_key, timeout=60)
print(inv_resp.headers['content-type'])
inv_resp.json()
inv_resp_code = inv_resp.status_code
print(inv_resp_code)
inv_resp_data=json.loads(inv_resp.text)

li_serials = []
li_labels = []
li_last_comm = []

#Parse out list of serials and meter names
for i in inv_resp_data['List']:
    li_serials.append(i['Serial'])
    li_labels.append(i['Label'])
    li_last_comm.append(i['LastCommSecUtc'])

li_serial_label_comm = list(zip(li_serials,li_labels,li_last_comm))
print('Meter Inventory API Called, List of Meters Compiled')

st = datetime.datetime.now()
li_resp = [] # Empty list; will hold list of dictionaries of JSON responses
jsons_df_inv = pd.DataFrame() # Empty dataframe to be populated with li_resp 

for serial in li_serials:
    #Format raw serial numbers with dash ("-") to create API URL
    serial = str(serial[0:3])+'-'+str(serial[3:]) 
    #Create API URL per meter
    api_url = 'https://api.eyedro.com/customcmd?Cmd=Unhcr.GetDeviceInventoryList&DeviceSerial='+str(serial)+'&UserKey=UNHCRMHiYgbHda9cRv4DuPp28DnAnfeV8s6umP5R'
    #Call API and dump results to JSON per Meter
    try:
        response = requests.get(api_url)
        response_data=json.loads(response.text)
        li_resp.append(response_data['List'][0])
        print(str(serial) + ' - Inventory Details Read Successfully')
    except Exception as e:
        print(str(serial)+ ' - Error: ' + str(e))

# Iterate through list of dictionaries, parse out information into cleaned format and append as new list of dicts

li_clean_dicts = []
dict_temp = {}

for i in li_resp:
    try: 
        dict_temp = {'Serial':i['Serial'],
                 'Label': i['Label'],
                 'LastCommSecUtc': i['LastCommSecUtc'], 
                 'LineFrequencyHz': i['LineFrequencyHz'],
                 'SensorA.AmpRating': i['SensorA.AmpRating'],
                 'SensorA.Voltage': i['SensorA.Voltage'],
                 'SensorB.AmpRating': i['SensorB.AmpRating'],
                 'SensorB.Voltage': i['SensorB.Voltage'],
                 'SensorC.AmpRating': i['SensorC.AmpRating'],
                 'SensorC.Voltage': i['SensorC.Voltage'],
                 'DisplayGroup': i['DisplayGroupList'][0]['Label'],
                 'SiteLabel' : i['SiteList'][0]['Label'],
                 'Latitude' : i['SiteList'][0]['Latitude'],
                 'Longitude' : i['SiteList'][0]['Longitude']}
        li_clean_dicts.append(dict_temp)
        print(f"{i['Serial']} - parsed")
    except:
        print(f"{i['Serial']} - failed")

# Roll together list of dicts into DF
df_jsons = pd.DataFrame.from_dict(li_clean_dicts)

# Dump DF to Excel
df_jsons.to_excel(output_full_path)

et = datetime.datetime.now()
print('Elapsed time: ' + str(et - st))

In [ ]:
### Old code from prior to March 8th API update

'''
print('--------------------------------')
import requests
import json
import datetime
import pandas as pd
from datetime import datetime
print('Libraries Imported')

inventory_url_base = "https://api.eyedro.com/customcmd?&Cmd=Unhcr.GetDeviceInventoryList&UserKey=" 
inventory_key = "UNHCRMHiYgbHda9cRv4DuPp28DnAnfeV8s6umP5R"
print('API URL and Key Set')

output_file = f"Meter_Inventory_Verbose{datetime.today().strftime('%Y-%m-%d')}.xlsx"

# Mac
output_path = '/Users/adamfifield/Library/CloudStorage/OneDrive-UNHCR/00 SharePoint Sites/Documents - Greening and Sustainability Team/Green Data Team/01 Data Sources/01 Greenbox/01 EyeDro API Data/03 Verbose Meter Details/'
output_full_path = output_path+output_file

# Windows
# output_path = r'C:\Users\FIFIELD\OneDrive - UNHCR\00 SharePoint Sites\Documents - Greening and Sustainability Team\Green Data Team\01 Data Sources\01 Greenbox\01 EyeDro API Data\03 Verbose Meter Details'
# output_full_path = output_path+'\\'+output_file


print('Output Path Set')
print('--------------------------------')

inv_resp = requests.get(inventory_url_base+inventory_key, timeout=60)
print(inv_resp.headers['content-type'])
inv_resp.json()
inv_resp_code = inv_resp.status_code
print(inv_resp_code)
inv_resp_data=json.loads(inv_resp.text)

li_serials = []
li_labels = []
li_last_comm = []

#Parse out list of serials and meter names
for i in inv_resp_data['List']:
    li_serials.append(i['Serial'])
    li_labels.append(i['Label'])
    li_last_comm.append(i['LastCommSecUtc'])

li_serial_label_comm = list(zip(li_serials,li_labels,li_last_comm))
print('Meter Inventory API Called, List of Meters Compiled')

st = datetime.now()
li_resp = [] # Empty list; will hold list of dictionaries of JSON responses
jsons_df_inv = pd.DataFrame() # Empty dataframe to be populated with li_resp 

for serial in li_serials:
    #Format raw serial numbers with dash ("-") to create API URL
    serial = str(serial[0:3])+'-'+str(serial[3:]) 
    #Create API URL per meter
    api_url = 'https://api.eyedro.com/customcmd?Cmd=Unhcr.GetDeviceInventoryList&DeviceSerial='+str(serial)+'&UserKey=UNHCRMHiYgbHda9cRv4DuPp28DnAnfeV8s6umP5R'
    #Call API and dump results to JSON per Meter
    try:
        response = requests.get(api_url)
        response_data=json.loads(response.text)
        li_resp.append(response_data['List'][0])
        print(str(serial) + ' - Inventory Details Read Successfully')
    except Exception as e:
        print(str(serial)+ ' - Error: ' + str(e))


df_jsons = pd.DataFrame.from_dict(li_resp) #Write list of dicts to DF

df_jsons.to_excel(output_full_path) #Dump DF to Excel

et = datetime.now()
print('Elapsed time: ' + str(et - st))

'''